In [5]:
# File Manipulation

import os # For working with Operating System
import sys # System arguments
from io import StringIO # String input/output
from dotenv import load_dotenv # Loading .env info

# Web

import requests # Accessing the Web

# Time

import datetime as dt # Working with dates/times
import pytz # Timezones
import time # For Sleeping

# Database 

import psycopg2
from psycopg2 import sql

# Data Manipulation

import numpy as np
import geopandas as gpd
import pandas as pd

### Load Functions

In [7]:
script_path = os.path.join('..', '..', 'Scripts', 'python')

# Function definition - Please see Scripts/python/*
exec(open(os.path.join(script_path, 'Get_spikes_df.py')).read())
exec(open(os.path.join(script_path, 'Create_messages.py')).read())
exec(open(os.path.join(script_path, 'twilio_functions.py')).read())
exec(open(os.path.join(script_path, 'Update_Alerts.py')).read())
exec(open(os.path.join(script_path, 'Send_Alerts.py')).read())
exec(open(os.path.join(script_path, 'Daily_Updates.py')).read())

### Global Variables

In [8]:
load_dotenv() # Load .env file

## API Keys

purpleAir_api = os.getenv('PURPLEAIR_API_TOKEN') # PurpleAir API Read Key

redCap_token_signUp = os.getenv('REDCAP_TOKEN_SIGNUP') # Survey Token
redCap_token_report = os.getenv('REDCAP_TOKEN_REPORT') # Report Token

## Database credentials

creds = [os.getenv('DB_NAME'),
         os.getenv('DB_USER'),
         os.getenv('DB_PASS'),
         os.getenv('DB_PORT'),
         os.getenv('DB_HOST')
        ]

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))  

## Twilio Information

TWILIO_ACCOUNT_SID = os.getenv('TWILIO_ACCOUNT_SID')
TWILIO_AUTH_TOKEN = os.getenv('TWILIO_AUTH_TOKEN')
TWILIO_NUMBER = os.getenv('TWILIO_NUMBER')

### When to Update?

In [9]:
# When to stop the program? (datetime)
days_to_run = 1 # How many days will we run this?
starttime = dt.datetime.now(pytz.timezone('America/Chicago')) 
stoptime = starttime + dt.timedelta(days=days_to_run)

In [21]:
print(starttime.replace(hour=8, minute = 0, second = 0) + dt.timedelta(days=1))

2023-11-14 08:00:00.440715-06:00
